# Анализ рисков и прибыли нефтедобывающей компании

[Загрузка и подготовка данных](#Загрузка-и-подготовка-данных)

[Обучение и проверка модели](#Обучение-и-проверка-модели)

[Подготовка к расчёту прибыли](#Подготовка-к-расчёту-прибыли)

[Расчёт прибыли и рисков](#Расчёт-прибыли-и-рисков)

# Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

def info(df): #Функция пробегает по признакам датафрейма и если тип у признака object, то смотрит уникальные значения, в противном случае оценивается распределение
    for column in df.columns:
        
        print(df[column].describe())
        print()
    
        if df[column].dtype == 'object':
            
            print(column)
            print(df[column].unique())
                    
        else:
            
            print(column)
            import plotly.express as px
            fig = px.histogram(df, x = column, marginal = 'box', title = 'Распределение '+ column)
            fig.show()
            
#info(geo_data_0)
#info(geo_data_1)
#info(geo_data_2)

In [2]:
print(geo_data_0.isnull().sum())
print(geo_data_1.isnull().sum())
print(geo_data_2.isnull().sum())
print(geo_data_0.duplicated().sum())
print(geo_data_1.duplicated().sum())
print(geo_data_2.duplicated().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
0
0
0


## Вывод

* Таблицы открыты и изучены. Все 3 таблицы имеют по 100000 строк и 5 столбцов. Пропущенных значений и дубликатов не обнаружено. По одному столбцу содержат данные типа object, остальные столбцы содержат данные типа float64. Оценили распределение некоторых признаков. 

# Обучение и проверка модели

In [3]:
# constants
state = np.random.RandomState(12345)

In [4]:
def train_model(data):
    target = data['product']
    features = data.drop(['product', 'id'] , axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    
    print('Важность признаков', model.coef_)
    
    predicted_valid = model.predict(features_valid) # получаем предсказания модели на валидационых данных
    predicted_valid_series = pd.Series(predicted_valid, index=features_valid.index)
    
    RMSE = np.sqrt(mean_squared_error(target_valid, predicted_valid)) # среднеквадратичная ошибка
    predicted_raw_oil_mean = predicted_valid.mean() # предсказанный средний объем запасов сырья в регионе
    raw_oil_mean = data['product'].mean() # средний объем запасов сырья в регионе по имеющимся данным
       
    print("RMSE =", RMSE)
    print("Средний запас предсказанного сырья:", predicted_raw_oil_mean)
    print("Средний запас сырья:", raw_oil_mean)
    
    bestDf = pd.DataFrame(predicted_valid) # создаем датафрейм с 2 колонками[prediction, product]
    bestDf.index = target_valid.index
    bestDf = bestDf.join(target_valid)
    
    top_200 = bestDf.sort_values(by=0, ascending=False)[:200] # сортируем по предсказанию и выбираем 200 лучших
    
    return [predicted_valid_series, target_valid, top_200]

In [5]:
for i in [geo_data_0, geo_data_1, geo_data_2]:
    train_model(i)
    print("\n")

Важность признаков [  3.59280585 -14.09794419   6.59316617]
RMSE = 37.5794217150813
Средний запас предсказанного сырья: 92.59256778438038
Средний запас сырья: 92.50000000000001


Важность признаков [-1.44940967e-01 -2.20278542e-02  2.69511805e+01]
RMSE = 0.893099286775616
Средний запас предсказанного сырья: 68.728546895446
Средний запас сырья: 68.82500000000002


Важность признаков [ 0.03093573 -0.04187719  5.70854243]
RMSE = 40.02970873393434
Средний запас предсказанного сырья: 94.96504596800489
Средний запас сырья: 95.00000000000004




## Вывод 

* Данные разделены на обучающую и валидационную выборки в соотношении 75:25;
* Модель обучена, сделаны предсказания на валидационной выборке;
* Предсказания на валидационной выборке и правильные ответы сохранены в отдельный датафрейм;
* На экран выведен средний запас предсказанного сырья и RMSE модели;
* Наименьшую среднеквадратичную ошибку имеет второй регион (geo_data_1);
* Наибольший запас предсказанного сырья имеет третий регион (geo_data_2);
* Наибольшей важностью обладает третий признак (f2).

# Подготовка к расчёту прибыли

In [6]:
# constants
one_product_price = 450000
regional_budget = 10000000000

In [7]:
no_loss_raw_oil_mean = regional_budget/one_product_price/200
print('Минимальный объем сырья для безубыточности:', round(no_loss_raw_oil_mean), 'тыс. баррелей')
for i in [geo_data_0, geo_data_1, geo_data_2]:
    product_mean = i["product"].mean()
    print('Средний запас в регионе равен', product_mean)

Минимальный объем сырья для безубыточности: 111 тыс. баррелей
Средний запас в регионе равен 92.50000000000001
Средний запас в регионе равен 68.82500000000002
Средний запас в регионе равен 95.00000000000004


## Вывод 

* Ключевые значения для расчётов сохранены в отдельных переменных;
* Достаточный объём сырья для безубыточной разработки новой скважины равен 111 тыс. баррелей. Средний запас сырья в каждом регионе меньше рассчитанного объема сырья для безубыточности. Применение ML для выбора лучших скважин в данном случае позволит сократить риски убытков.

# Расчёт прибыли и рисков 

In [8]:
def top_200_profit(df_top_200):
    income = df_top_200['product'].sum()*one_product_price
    profit = income - regional_budget
    print('Доход в регионе:', income)
    print('Прибыль в регионе:', profit)
    return [income, profit]

In [9]:
def best_oil_hole_profit (target, probabilities, count): # суммарная прибыль с 200 лучших отобранных скважин
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    profit = (selected.sum() * one_product_price - regional_budget)
    return profit

In [10]:
def bootstrap(predictions, target):
    values = []
    loss = 0
    
    for i in range(1000):
        target_sample = target.sample(n = 500, replace=True, random_state=state) # выборка рандомных объемов 500 скважин 
        probs_sample = predictions[target_sample.index] # отбираем по индексу предсказанные объемы запасов сырья в регионе для 500 выбранных ранее скважин
        profit = best_oil_hole_profit(target_sample, probs_sample, 200) # считаем прибыльность 200 лучших отобранных скважин
        values.append(profit)
        if profit < 0:
            loss += 1
     
    values = pd.Series(values)
    risk = (loss / len(values))*100
    upper_border = values.quantile(0.975)
    lower_border = values.quantile(0.025)
    
    print("Средняя прибыль отобранных месторождений", values.mean())
    print("Верхняя граница доверительного интервала", upper_border)
    print("Нижняя граница доверительного интервала", lower_border)
    print("Риск убытков в регионе", risk)

In [11]:
for i in [geo_data_0, geo_data_1, geo_data_2]:
    predicted_valid_series, target_valid, top_200 = train_model(i)
    top_200_profit(top_200)
    bootstrap(predicted_valid_series, target_valid)
    print("\n")

Важность признаков [  3.59280585 -14.09794419   6.59316617]
RMSE = 37.5794217150813
Средний запас предсказанного сырья: 92.59256778438038
Средний запас сырья: 92.50000000000001
Доход в регионе: 13320826043.139853
Прибыль в регионе: 3320826043.1398525
Средняя прибыль отобранных месторождений 425938526.9105923
Верхняя граница доверительного интервала 947976353.358369
Нижняя граница доверительного интервала -102090094.83793654
Риск убытков в регионе 6.0


Важность признаков [-1.44940967e-01 -2.20278542e-02  2.69511805e+01]
RMSE = 0.893099286775616
Средний запас предсказанного сырья: 68.728546895446
Средний запас сырья: 68.82500000000002
Доход в регионе: 12415086696.68151
Прибыль в регионе: 2415086696.681511
Средняя прибыль отобранных месторождений 518259493.69732493
Верхняя граница доверительного интервала 953612982.0669085
Нижняя граница доверительного интервала 128123231.43308629
Риск убытков в регионе 0.3


Важность признаков [ 0.03093573 -0.04187719  5.70854243]
RMSE = 40.029708733934

## Вывод

* Применена техника Bootstrap для расчета распределения прибыли по регионам;
* Лучший регион по средней прибыли отобранных месторождений geo_data_1 (около 517 млн.руб.);
* Худший регион по средней прибыли отобранных месторождений geo_data_2 (около 412 млн.руб.);
* Риск убытков регионов geo_data_0 и geo_data_2 более чем в 2 раза превышают допустимые 2,5%. Данные регионы не подходят для разработки;
* Риск убытков в регионе geo_data_1 составляет 1,2%, прогнозируемая прибыль около 517 млн.руб. - регион подходит для ведения разработки месторождений.